In [ ]:
# here we will be using NMF (non negative matrix factorization) for topic modelling

In [ ]:
import pandas as pd
import numpy as np
from sklearn import decomposition
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk import SnowballStemmer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true',compression='zip')
data.head()

In [ ]:
data.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [ ]:
data = data[['Consumer complaint narrative','Product','Company']].rename(columns={'Consumer complaint narrative':'complaints'})
data.head()

,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

In [ ]:
pd.set_option('display.max_colwidth', -1)
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

In [ ]:
data.Company.value_counts()

CITIBANK, N.A.                           3226
CAPITAL ONE FINANCIAL CORPORATION        2711
BANK OF AMERICA, NATIONAL ASSOCIATION    2580
JPMORGAN CHASE & CO.                     2409
WELLS FARGO & COMPANY                    2001
                                         ... 
Couch Conville & Blitt LLC               1   
Creditors Collection Bureau, Inc.        1   
Essential Retrieval Group                1   
Park View Credit                         1   
K Street Finance, Inc.                   1   
Name: Company, Length: 2197, dtype: int64

In [ ]:
data.Product.value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                       9799 
Checking or savings account    7003 
Student loan                   2950 
Vehicle loan or lease          2736 
Name: Product, dtype: int64

In [ ]:
stemmer = SnowballStemmer('english')


In [ ]:
# preprocess each complaints
stopwords = set(nltk.corpus.stopwords.words('english'))

def tokenize(text):
  token = [str(token) for token in nltk.word_tokenize(text) if( len(token)>3 and len(token.strip('Xx/'))>2)]
  stems = [stemmer.stem(item) for item in token if (item not in stopwords)]
  return token

In [ ]:
vector_tf = TfidfVectorizer(tokenizer=tokenize, stop_words=None, max_df=.75, min_df=50, use_idf=True, ngram_range=(1,2), max_features=1000)
tf = vector_tf.fit_transform(data['complaints'])


In [ ]:
tf.A.shape

(57453, 1000)

In [ ]:
vector_tf.get_feature_names()

['100.00',
 '1000.00',
 '200.00',
 '500.00',
 'ability',
 'able',
 'about',
 'about this',
 'above',
 'accept',
 'accepted',
 'access',
 'accordance',
 'accordance with',
 'according',
 'account',
 'account closed',
 'account from',
 'account have',
 'account number',
 'account that',
 'account they',
 'account with',
 'accounts',
 'accurate',
 'action',
 'actions',
 'activity',
 'actual',
 'actually',
 'added',
 'addition',
 'additional',
 'address',
 'advised',
 'advised that',
 'affidavit',
 'afford',
 'after',
 'again',
 'against',
 'agencies',
 'agency',
 'agent',
 'agreed',
 'agreement',
 'alleged',
 'alleged debt',
 'allow',
 'allowed',
 'almost',
 'along',
 'along with',
 'already',
 'also',
 'although',
 'always',
 'america',
 'american',
 'american express',
 'amex',
 'amount',
 'amounts',
 'annual',
 'another',
 'answer',
 'anyone',
 'anything',
 'application',
 'applied',
 'apply',
 'appraisal',
 'approved',
 'approximately',
 'around',
 'asked',
 'asked them',
 'asking',
 

In [ ]:
nmf = decomposition.NMF(n_components=6, random_state=11)
w1 = nmf.fit_transform(tf)

In [ ]:
w1.shape

(57453, 6)

In [ ]:
tf.A[0].shape

(1000,)

In [ ]:
len(vector_tf.get_feature_names())

1000

In [ ]:
# probability of document reating to a particular topic

w1[0] 

array([0.0095878 , 0.02415905, 0.0199274 , 0.0176434 , 0.00374499,
       0.00344008])

In [ ]:
h1 = nmf.components_
h1.shape

# each topic and probabilty of words related to each topic

(6, 1000)

In [ ]:
h1[0]

In [ ]:
vocab = np.array(vector_tf.get_feature_names())
get_words = lambda topic: [vocab[word] for word in np.argsort(topic)[:-15-1:-1]]
words_per_topic = [get_words(topic) for topic in h1]
topics = [', '.join(word) for word in words_per_topic]

In [ ]:
topics

['they, that, them, have, told, call, called, said, would, with, phone, back, from, that they, asked',
 'debt, this, credit, collection, report, credit report, company, this debt, reporting, have, that, from, with, account, information',
 'account, bank, check, that, this, bank america, america, checking, funds, money, with, closed, checking account, chase, deposit',
 'loan, payment, mortgage, payments, that, late, interest, have, this, with, loans, made, been, paid, home',
 'identity theft, theft, identity, victim, belong, affidavit, report, attached, does, legal, this debt, please, this, debt, this account',
 'card, credit, credit card, capital, charges, charge, citi, balance, that, cards, late, limit, this, interest, report']